In [11]:
import os
from unstructured.partition.pdf import partition_pdf

In [12]:
dress_code_pdf_path = 'pdfs/Dress Code Policy.pdf'
promotion_pdf_path = 'pdfs/Promotion Policy - 2025.pdf'

In [13]:
from pdf2image import convert_from_path
from PIL import Image, ImageFilter
import pytesseract

images = convert_from_path(promotion_pdf_path, dpi=300)

for i, img in enumerate(images):
    img = img.convert("L").filter(ImageFilter.SHARPEN)  # grayscale + sharpen
    text = pytesseract.image_to_string(img)
    print(f"Page {i+1}:", text[:500])


Page 1: ELSEWEDY
ELECTRIC
a

ELSEWEDY ELECTRIC GROUP

Promotion Policy

www.elsewedyelectric.com

Page 2: ELSEWEDY

ELECTRIC
a

Promotion Policy

Purpose
To set, define and communicate broadly Elsewedy Electric policy concerning promotions

ensuring equal opportunity to all employees based on job evolution, performance, and

business need.

This Policy is a minimum standard; where local legislations define higher standards; the Group

shall comply with them.

Applicability
This policy applies to all the operating companies and subsidiaries directly or indirectly

controlled by Elsewedy Electric, and
Page 3: ELSEWEDY

ELECTRIC
=[—[——

The 9-Box Grid of Talent Differentiation

The 9-Box Grid of Talent Differentiation is a tool used during the TRM (Talent Review Meeting)
to measure the (potentiality) of the employees in comparisons to each other (within the same
band) based on the 3 last year’s performance and capabilities scores.

The output of the grid is used to take the Talent Managem

In [14]:
elements = partition_pdf(
    promotion_pdf_path,
    strategy="hi_res",
)
elements

In [15]:
def group_semantic_chunks(elements, file_path):
    chunks = []
    current_title = "Untitled Section"
    current_content = []

    for el in elements:
        print(f"{el.category} :: {el.text}")

        # Skip headers and footers
        if el.category in ["Header", "Footer"]:
            continue

        text = el.text.strip()
        if not text:
            continue

        # Start new chunk when a title appears
        if el.category == "Title":
            # Save current chunk before starting new one
            if current_content:
                chunks.append({
                    "title": current_title,
                    "content": "\n".join(current_content),
                    "source": os.path.basename(file_path),
                })
                current_content = []
            current_title = text  # update title
        else:
            current_content.append(text)

    # Add the last chunk
    if current_content:
        chunks.append({
            "title": current_title,
            "content": "\n".join(current_content),
            "source": os.path.basename(file_path),
        })

    return chunks


In [16]:
semantic_chunks = group_semantic_chunks(elements=elements, file_path=promotion_pdf_path)

Image :: ELSEWEDY ELECTRIC ee
Title :: ELSEWEDY ELECTRIC GROUP
NarrativeText :: Promotion Policy
Image :: 
NarrativeText :: www.elsewedyelectric.com
UncategorizedText :: 1
Image :: ELSEWEDY ELECTRIC ————
Title :: Promotion Policy
Title :: Purpose
NarrativeText :: To set, define and communicate broadly Elsewedy Electric policy concerning promotions ensuring equal opportunity to all employees based on job evolution, performance, and business need.
NarrativeText :: This Policy is a minimum standard; where local legislations define higher standards; the Group shall comply with them.
Title :: Applicability
NarrativeText :: This policy applies to all the operating companies and subsidiaries directly or indirectly controlled by Elsewedy Electric, and all the geographical regions where Elsewedy Electric companies and subsidiaries are operating.
UncategorizedText :: The Performance and Talent Cycle
Image :: €LSEWEDY €LECTRIC —
UncategorizedText :: raining Calendar
UncategorizedText :: 8. it Lea

In [17]:
semantic_chunks[1]['content']

'Promotion Policy\nwww.elsewedyelectric.com\n1\nELSEWEDY ELECTRIC ————'